# Introduction

## Imports

In [1]:
from datetime import datetime
import itertools
import json
import os
import random
import shutil
import sys
import time

In [2]:
from data.InstagramAPI.InstagramAPI import InstagramAPI
from data.plots import plot_confusion_matrix

Fail to import moviepy. Need only for Video upload.


In [7]:
from imutils import paths #pip install opencv-python
from sklearn import metrics, model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.applications import VGG16, imagenet_utils
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import tensorflow as tf

### Install Packages

In [ ]:
#On Anaconda Prompt:
#pip install tensorflow

In [ ]:
pip install requests_toolbelt

In [6]:
pip install pandas

  Using cached pandas-1.3.0-cp39-cp39-win_amd64.whl (10.2 MB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install matplotlib

In [ ]:
pip install sklearn

In [ ]:
pip install imutils

In [ ]:
pip install opencv-python

## Configs

In [ ]:
TYPE = "guastafriends"

DATE_BEGIN = datetime(2021,1,1,0,0).timestamp()

PATH_BASE                   = os.getcwd()
PATH_BASE_DATA              = os.path.join(PATH_BASE, "data")
PATH_1_CREATING_DATASET     = os.path.join(PATH_BASE_DATA, "1-Creating_Dataset")
PATH_FIRST_DATASET          = os.path.join(PATH_1_CREATING_DATASET, "Dataset")
PATH_FIRST_DATASET_TYPE     = os.path.join(PATH_FIRST_DATASET, TYPE)
PATH_2_PREPARING_DATASET    = os.path.join(PATH_BASE_DATA, "2-Preparing_Dataset")
PATH_CLEAN_DATASET          = os.path.join(PATH_2_PREPARING_DATASET, "Clean_Dataset")
PATH_CLEAN_DATASET_TYPE     = os.path.join(PATH_CLEAN_DATASET, TYPE)
PATH_3_PROCESSING_DATASET   = os.path.join(PATH_BASE_DATA, "3-Processing_Dataset")
PATH_SIMILARY_ACCOUNTS      = os.path.join(PATH_3_PROCESSING_DATASET, "Similary_Accounts")
PATH_SIMILARY_ACCOUNTS_TYPE = os.path.join(PATH_SIMILARY_ACCOUNTS, TYPE)
PATH_LABEL_COUNTER          = os.path.join(PATH_3_PROCESSING_DATASET, "Label_Counter")
PATH_LABEL_COUNTER_TYPE     = os.path.join(PATH_LABEL_COUNTER, TYPE)
PATH_4_CLASSIFYING_DATASET  = os.path.join(PATH_BASE_DATA, "4-Classifying_Dataset")

MEDIA_TYPES = [1] #8
CANDIDATE = -1
#CANDIDATE = 0
#CANDIDATE = 1
#CANDIDATE = 2

ONLY_WITH_COMMENTS = True

#FOLLOWERS_RADIUS = 3
#LIKES_RADIUS = 95

FOLLOWERS_RADIUS = 100
LIKES_RADIUS = 0

RANDOM_IGUAL = True
#RANDOM_IGUAL = False

BATCH_LABEL_COUNTER = 7500

BATCH_SIZE = 60
CV = 10

print(f"DATE_BEGIN                  : {DATE_BEGIN}")
print(f"PATH_BASE                   : {PATH_BASE}")
print(f"PATH_BASE_DATA              : {PATH_BASE_DATA}")
print(f"PATH_1_CREATING_DATASET     : {PATH_1_CREATING_DATASET}")
print(f"PATH_FIRST_DATASET          : {PATH_FIRST_DATASET}")
print(f"PATH_FIRST_DATASET_TYPE     : {PATH_FIRST_DATASET_TYPE}")
print(f"PATH_2_PREPARING_DATASET    : {PATH_2_PREPARING_DATASET}")
print(f"PATH_CLEAN_DATASET          : {PATH_CLEAN_DATASET}")
print(f"PATH_CLEAN_DATASET_TYPE     : {PATH_CLEAN_DATASET_TYPE}")
print(f"PATH_3_PROCESSING_DATASET   : {PATH_3_PROCESSING_DATASET}")
print(f"PATH_SIMILARY_ACCOUNTS      : {PATH_SIMILARY_ACCOUNTS}")
print(f"PATH_SIMILARY_ACCOUNTS_TYPE : {PATH_SIMILARY_ACCOUNTS_TYPE}")
print(f"PATH_LABEL_COUNTER          : {PATH_LABEL_COUNTER}")
print(f"PATH_LABEL_COUNTER_TYPE     : {PATH_LABEL_COUNTER_TYPE}")
print(f"PATH_4_CLASSIFYING_DATASET  : {PATH_4_CLASSIFYING_DATASET}")

## Global Functions

In [ ]:
def write_file(path_file, data, access_mode='a'):
    file = open(path_file, access_mode)
    if isinstance(data, list):
        for d in data:
            file.write(d)
    else:
        file.write(data)
    file.close()
    
def read_file(path_file, access_mode='r'):
    file = open(path_file, access_mode)
    data = file.readlines()
    file.close()  
    return data

def write_json(json_path, data, access_mode='w', encoding="utf-8"):
    with open(json_path, access_mode, encoding=encoding) as json_file:
        json.dump(data, json_file, indent=4, ensure_ascii=False)

def read_json(json_path, access_mode='r', encoding="utf-8"):
    with open(json_path, access_mode, encoding=encoding) as json_file:
        data = json.load(json_file)
    return data

##############################################

def get_accounts():
    list_accounts = read_file(os.path.join(PATH_1_CREATING_DATASET, f"{TYPE}_users.txt"))
    return [s.strip() for s in list_accounts]

def log_print(data):
    print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - {data}", flush=True)

def create_path(path):
    if not os.path.exists(path):
        os.makedirs(path)
        log_print(f"Path '{path}' was created.")

def delete_path(path):
    if os.path.exists(path):
        shutil.rmtree(path, ignore_errors=True)
        log_print(f"Path '{path}' was deleted.")

def random_sleep(min=0.1, max=5):
    time.sleep(round(random.uniform(min, max), 1))

##############################################

def loop_images_extract_features(imagePaths, labels, csv):
    # loop over the images in batches
    for (b, i) in enumerate(range(0, len(imagePaths), BATCH_SIZE)):
        # extract the batch of images and labels, then initialize the
        # list of actual images that will be passed through the network
        # for feature extraction
        log_print("[INFO] processing batch {}/{}".format(b + 1, int(np.ceil(len(imagePaths) / float(BATCH_SIZE)))))
        batchPaths = imagePaths[i:i + BATCH_SIZE]
        batchLabels = le.transform(labels[i:i + BATCH_SIZE])
        batchImages = []

        # loop over the images and labels in the current batch
        for imagePath in batchPaths:
            # load the input image using the Keras helper utility while ensuring the image is resized to 224x224 pixels
            image = load_img(imagePath, target_size=(224, 224))
            image = img_to_array(image)

            # preprocess the image by (1) expanding the dimensions and
            # (2) subtracting the mean RGB pixel intensity from the ImageNet dataset
            image = np.expand_dims(image, axis=0)
            image = imagenet_utils.preprocess_input(image)

            # add the image to the batch
            batchImages.append(image)

        # pass the images through the network and use the outputs as
        # our actual features, then reshape the features into a
        # flattened volume
        batchImages = np.vstack(batchImages)
        features = model.predict(batchImages, batch_size=BATCH_SIZE)
        features = features.reshape((features.shape[0], 7 * 7 * 512))

        # loop over the class labels and extracted features
        for (label, vec) in zip(batchLabels, features):
            # construct a row that exists of the class label and
            # extracted features
            vec = ",".join([str(v) for v in vec])
            csv.write(f"{label},{vec}\n")
            
def load_data_split(splitPath):
    # initialize the data and labels
    data = []
    labels = []

    # loop over the rows in the data split file
    for row in open(splitPath):
        # extract the class label and features from the row
        row = row.strip().split(",")
        label = row[0]
        features = np.array(row[1:], dtype="float")

        # update the data and label lists
        data.append(features)
        labels.append(label)

    # convert the data and labels to NumPy arrays
    data = np.array(data)
    labels = np.array(labels)

    # return a tuple of the data and labels
    return (data, labels)

### Reset Type

In [ ]:
delete_path(PATH_FIRST_DATASET_TYPE)
delete_path(PATH_CLEAN_DATASET_TYPE)
delete_path(PATH_SIMILARY_ACCOUNTS_TYPE)
delete_path(PATH_LABEL_COUNTER_TYPE)

### Checking Folders

In [ ]:
#create_path(PATH_BASE_DATA)
#create_path(PATH_1_CREATING_DATASET)
#create_path(PATH_FIRST_DATASET)
create_path(PATH_FIRST_DATASET_TYPE)
#create_path(PATH_2_PREPARING_DATASET)
#create_path(PATH_CLEAN_DATASET)
create_path(PATH_CLEAN_DATASET_TYPE)
#create_path(PATH_3_PROCESSING_DATASET)
#create_path(PATH_SIMILARY_ACCOUNTS)
create_path(PATH_SIMILARY_ACCOUNTS_TYPE)
#create_path(PATH_LABEL_COUNTER)
create_path(PATH_LABEL_COUNTER_TYPE)
create_path(PATH_4_CLASSIFYING_DATASET)

# Creating Dataset

## Getting Data

In [ ]:
file_logins = read_json(os.path.join(PATH_1_CREATING_DATASET, "logins.json"))

def error_log(acc, log_type, e):
    log_print(f"Failed to process {acc} account: {str(e)}")
    log_print(f"Appending to file {TYPE}_users_error.log")
    write_file(os.path.join(PATH_1_CREATING_DATASET, f"{TYPE}_users_error.log"), \
               [f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - {log_type} - {acc} - {str(e)}", "\n"])
    print("**********************************************\n")
        
def get_login(id_login=0, logins=file_logins["Logins"]):
    api = None
    logged = False
    while not logged:      
        log_print("Trying login in account: " + logins[id_login]["username"])
        api = InstagramAPI(logins[id_login]["username"], logins[id_login]["password"])
        logged = api.login()
        if not logged:
            id_login += 1
            if id_login >= len(logins):
                id_login = 0
            random_sleep()
    return (api, id_login)
        
def search_username(api, id_login, acc):
    log_print(f"Searching username {acc}")

    while True:
        
        api.searchUsername(acc)

        log_print(f"searchUsername-status_code: {api.LastResponse.status_code}")
        if api.LastResponse.status_code == 429 or ('message' in api.LastJson and api.LastJson["message"] == 'challenge_required'):
            (api, id_login) = get_login(id_login=id_login+1)
        elif api.LastResponse.status_code != 200:
            log_print(f"LastResponse: {api.LastResponse}")
            log_print(f"LastJson: {api.LastJson}")
            raise Exception(f"{api.LastResponse.status_code}-{api.LastJson}")
        else:
            break
        
        random_sleep()
    
    return (api, id_login)

def create_file_user_info(api, id_login, acc):
    
    (api, id_login) = search_username(api, id_login, acc)
    
    imgs_user = {}
    imgs_user["user"] = acc
    imgs_user["downloaded"] = 0

    create_path(os.path.join(PATH_FIRST_DATASET_TYPE, acc))
    user = api.LastJson["user"]
    
    log_print("Creating file user_info")
    user["collected_at"] = datetime.timestamp(datetime.now())
    
    write_json(os.path.join(PATH_FIRST_DATASET_TYPE, acc, "user_info.json"), user)
    log_print("File user_info created")
    
    return (api, id_login, user)

def get_user_feed(api, id_login, user, next_max_id=None):  
    
    while True:   
        if next_max_id == None:
            api.getUserFeed(user['pk'], minTimestamp=int(DATE_BEGIN))
        else:
            api.getUserFeed(user['pk'], maxid=next_max_id, minTimestamp=int(DATE_BEGIN))

        log_print(f"getUserFeed-status_code: {api.LastResponse.status_code}")
        if api.LastResponse.status_code == 429 or ('message' in api.LastJson and api.LastJson["message"] == 'challenge_required'):
            (api, id_login) = get_login(id_login=id_login+1)
        elif api.LastResponse.status_code != 200:
            log_print(f"LastResponse: {api.LastResponse}")
            log_print(f"LastJson: {api.LastJson}")
            raise Exception(f"{api.LastResponse.status_code}-{api.LastJson}")
        else:
            break
            
        random_sleep()
    
    return (api, id_login)

def create_file_user_posts(api, id_login, user):
    log_print("Creating file user_posts")
    (api, id_login) = get_user_feed(api, id_login, user)
        
    posts = api.LastJson

    while (posts["more_available"]):
        random_sleep()
        aux_bool = False
        (api, id_login) = get_user_feed(api, id_login, user, posts["next_max_id"])

        posts["items"].extend(api.LastJson["items"])
        posts["num_results"] += api.LastJson["num_results"]
        posts["more_available"] =  api.LastJson["more_available"]
        if posts["more_available"]:
            posts["next_max_id"] = api.LastJson["next_max_id"]
        else:
            del posts['next_max_id']
        posts["auto_load_more_enabled"] = api.LastJson["auto_load_more_enabled"]
        posts["status"] = api.LastJson["status"]
        
    posts["collected_at"] = datetime.timestamp(datetime.now())
    write_json(os.path.join(PATH_FIRST_DATASET_TYPE, acc, "user_posts.json"), posts)
    log_print("File user_posts created")
    
    return (api, id_login, posts)

In [ ]:
if not os.path.exists(os.path.join(PATH_1_CREATING_DATASET, f"{TYPE}_users.bkp")):
    shutil.copyfile(os.path.join(PATH_1_CREATING_DATASET, f"{TYPE}_users.txt"), os.path.join(PATH_1_CREATING_DATASET, f"{TYPE}_users.bkp"))

(api, id_login) = get_login()

log_print("Type: " + TYPE)
log_print("Starting...\n")

list_accounts = get_accounts()
for acc in list_accounts:
    try:
        
        (api, id_login, user) = create_file_user_info(api, id_login, acc)
        (api, id_login, posts) = create_file_user_posts(api, id_login, user)
            
    except Exception as e:
        error_log(acc, "GettingData", e)
    else:
        log_print(f"Account {acc} successfully processed")
        print("**********************************************\n")

log_print("Finished")

## Creating Overview

In [ ]:
def create_file_user_posts_overview(posts):
    
    log_print("Creating file user_posts_overview")
    
    media_count_total = 0
    posts_overview = {}
    posts_overview["collected_at"] = posts["collected_at"]
    posts_overview["num_results"] = posts["num_results"]
    posts_overview["items"] = []
    
    for post in posts["items"]:
        
        if not post["media_type"] in MEDIA_TYPES:
            continue
        if ONLY_WITH_COMMENTS and "comments_disabled" in post:
            continue
            
        post_overview = {}
        post_overview["pk"] = post["pk"]
        post_overview["id"] = post["id"]
        post_overview["taken_at"] = post["taken_at"]
        post_overview["taken_at_year"] = datetime.fromtimestamp(post_overview["taken_at"]).year

        post_overview["caption_text"] = ""
        if not post["caption"] is None:
            post_overview["caption_text"] = post["caption"]["text"]

        post_overview["media_type"] = post["media_type"]
        post_overview["images"] = []
        if post_overview["media_type"] == 1:
            post_overview["media_count"] = 1
            
            candidate_aux = CANDIDATE
            while candidate_aux >= len(post["image_versions2"]["candidates"]):
                candidate_aux -= 1
                
            image = {}
            image["pk"] = post_overview["pk"]
            image["width"] = post["image_versions2"]["candidates"][candidate_aux]["width"]
            image["height"] = post["image_versions2"]["candidates"][candidate_aux]["height"]
            image["url"] = post["image_versions2"]["candidates"][candidate_aux]["url"]
            post_overview["images"].append(image)
        elif post_overview["media_type"] == 8:
            post_overview["media_count"] = post["carousel_media_count"]
            for carousel_post in post["carousel_media"]:
                
                candidate_aux = CANDIDATE
                while candidate_aux >= len(carousel_post["image_versions2"]["candidates"]):
                    candidate_aux -= 1
                
                image = {}
                image["pk"] = carousel_post["pk"]
                image["width"] = carousel_post["image_versions2"]["candidates"][candidate_aux]["width"]
                image["height"] = carousel_post["image_versions2"]["candidates"][candidate_aux]["height"]
                image["url"] = carousel_post["image_versions2"]["candidates"][candidate_aux]["url"]
                post_overview["images"].append(image)

        media_count_total += post_overview["media_count"]

        post_overview["location"] = None
        if "location" in post:
            post_overview["location"] = post["location"]

        post_overview["like_count"] = post["like_count"]

        post_overview["comments_disabled"] = False
        if "comments_disabled" in post:
            post_overview["comments_disabled"] = post["comments_disabled"]
        if post_overview["comments_disabled"]:
            post_overview["comment_count"] = 0
        else:
            post_overview["comment_count"] = post["comment_count"]

        post_overview["caption_text_length"] = len(post_overview["caption_text"])
        hashtags = [word[1:] for word in post_overview["caption_text"].split() if word[0] == '#']
        post_overview["caption_hashtags_count"] = len(hashtags)
        post_overview["caption_hashtags"] = []

        if post_overview["caption_hashtags_count"] > 0:
            post_overview["caption_hashtags"] = hashtags
        post_overview["timestamp_duration"] = posts_overview["collected_at"] - post_overview["taken_at"]
        post_overview["likes_by_duration"] = post_overview["like_count"] / post_overview["timestamp_duration"]
        post_overview["comments_by_duration"] = post_overview["comment_count"] / post_overview["timestamp_duration"]

        posts_overview["items"].append(post_overview)
            
    posts_overview["media_count_total"] = media_count_total
    write_json(os.path.join(PATH_FIRST_DATASET_TYPE, acc, "user_posts_overview.json"), posts_overview)
    log_print("File user_posts_overview created")

In [ ]:
list_accounts = get_accounts()
for acc in list_accounts:
    
    try:
        log_print(f"Processing account {acc}")

        posts = read_json(os.path.join(PATH_FIRST_DATASET_TYPE, acc, "user_posts.json"))

        create_file_user_posts_overview(posts)

    except Exception as e:
        error_log(acc, "CreatingOverview", e)
    else:
        log_print(f"Account {acc} successfully processed")
        print("**********************************************\n")
        
log_print("Finished")

## Downloading Images

In [ ]:
def download_image(url, path):
    resp = requests.get(url, stream=True, timeout=20)  # Open the url image, set stream to True, this will return the stream content.
    local_file = open(path, 'wb')  # Open a local file with wb ( write binary ) permission.
    resp.raw.decode_content = True  # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
    shutil.copyfileobj(resp.raw, local_file)  # Copy the response stream raw data to local image file.

In [ ]:
again = True
while again:
    again = False
    list_accounts = get_accounts()
    for acc in list_accounts:

        try:
            log_print(f"Processing account {acc}")
            
            create_path(os.path.join(PATH_FIRST_DATASET_TYPE, acc, "images"))
                
            posts_overview = read_json(os.path.join(PATH_FIRST_DATASET_TYPE, acc, "user_posts_overview.json"))

            log_print("Downloading posts pictures:")
            imgs_user_downloaded = len(list(next(os.walk(os.path.join(PATH_FIRST_DATASET_TYPE, acc, "images")))[2])) - 1 #0 - root, 1 - dirs, 2 - files
            if imgs_user_downloaded <= 0:
                imgs_user_downloaded = 0
            if imgs_user_downloaded < posts_overview['media_count_total']:
                for post in posts_overview["items"]:
                    for image in post["images"]:
                        if not os.path.exists(os.path.join(PATH_FIRST_DATASET_TYPE, acc, "images", str(image["pk"]) + ".jpg")):
                            download_image(image['url'], os.path.join(PATH_FIRST_DATASET_TYPE, acc, "images", f"{image['pk']}.jpg"))
                            imgs_user_downloaded += 1
                            log_print(f"{imgs_user_downloaded}/{posts_overview['media_count_total']}")
                
            log_print(f"{imgs_user_downloaded} image(s) downloaded")
        except Exception as e:
            error_log(acc, "DownloadingImages", e)
            again = True
        else:
            log_print(f"Account {acc} successfully processed")
            print("**********************************************\n")
            
log_print("Finished")

# Preparing Dataset

1 - Depois de executar "Checking Folders":
    Dentro da pasta "CNN/CNN_Dataset/[nicho]" você precisa inserir imagens para cada classe (pastas [nicho] e non_[nicho], por exemplo, dog e non_dog) dentro de cada pasta "Train" e "Test".
    Infelizmente você precisa buscar de maneira manual imagens que pertencem e não pertencem ao nicho que você está estudando.
    Recomenda-se pelo menos 1500 imagens para cada classe para o treinamento, totalizando 3000 imagens para o treinamento, e 500 imagens para cada classe para o teste, totalizando 1000 imagens para o teste.

2 - O próximo passo é executar 3-Renaming_Images.py, para renomear cada imagem automaticamente, preparando-as para a CNN.

3 - Executar 4-Extract_Features.py para extrair as características do conjunto de dados.

4 - Executar 5-Train.py para treinar a CNN.

5 - Com a CNN treinada, executar 6-Clean.py para classificar/limpar o conjunto de imagens do Instagram.

6 - Por fim, executar 7-Moving_Images.py, para mover as imagens para o dataset limpo, em Clean_Dataset.

7 - O Arquivo 8-Statistics.py trás algumas estatísticas sobre o dataset limpo.

## CNN Clean Configs

In [ ]:
PATH_CNN_CLEAN_BASE                       = os.path.join(PATH_2_PREPARING_DATASET, "CNN_Clean")
PATH_CNN_CLEAN_DATASET                    = os.path.join(PATH_CNN_CLEAN_BASE, "CNN_Dataset")
PATH_CNN_CLEAN_DATASET_TYPE               = os.path.join(PATH_CNN_CLEAN_DATASET, TYPE)
PATH_CNN_CLEAN_DATASET_TYPE_TRAIN         = os.path.join(PATH_CNN_CLEAN_DATASET_TYPE, "Train")
PATH_CNN_CLEAN_DATASET_TYPE_TRAIN_TYPE    = os.path.join(PATH_CNN_CLEAN_DATASET_TYPE_TRAIN, TYPE)
PATH_CNN_CLEAN_DATASET_TYPE_TRAIN_NONTYPE = os.path.join(PATH_CNN_CLEAN_DATASET_TYPE_TRAIN, f"non_{TYPE}")
PATH_CNN_CLEAN_DATASET_TYPE_TEST          = os.path.join(PATH_CNN_CLEAN_DATASET_TYPE, "Test")
PATH_CNN_CLEAN_DATASET_TYPE_TEST_TYPE     = os.path.join(PATH_CNN_CLEAN_DATASET_TYPE_TEST, TYPE)
PATH_CNN_CLEAN_DATASET_TYPE_TEST_NONTYPE  = os.path.join(PATH_CNN_CLEAN_DATASET_TYPE_TEST, f"non_{TYPE}")
PATH_CNN_CLEAN_OUTPUT                     = os.path.join(PATH_CNN_CLEAN_BASE, "CNN_Output")
PATH_CNN_CLEAN_OUTPUT_TYPE                = os.path.join(PATH_CNN_CLEAN_OUTPUT, TYPE)

print(f"PATH_CNN_CLEAN_BASE                      : {PATH_CNN_CLEAN_BASE}")
print(f"PATH_CNN_CLEAN_DATASET                   : {PATH_CNN_CLEAN_DATASET}")
print(f"PATH_CNN_CLEAN_DATASET_TYPE              : {PATH_CNN_CLEAN_DATASET_TYPE}")
print(f"PATH_CNN_CLEAN_DATASET_TYPE_TRAIN        : {PATH_CNN_CLEAN_DATASET_TYPE_TRAIN}")
print(f"PATH_CNN_CLEAN_DATASET_TYPE_TRAIN_TYPE   : {PATH_CNN_CLEAN_DATASET_TYPE_TRAIN_TYPE}")
print(f"PATH_CNN_CLEAN_DATASET_TYPE_TRAIN_NONTYPE: {PATH_CNN_CLEAN_DATASET_TYPE_TRAIN_NONTYPE}")
print(f"PATH_CNN_CLEAN_DATASET_TYPE_TEST         : {PATH_CNN_CLEAN_DATASET_TYPE_TEST}")
print(f"PATH_CNN_CLEAN_DATASET_TYPE_TEST_TYPE    : {PATH_CNN_CLEAN_DATASET_TYPE_TEST_TYPE}")
print(f"PATH_CNN_CLEAN_DATASET_TYPE_TEST_NONTYPE : {PATH_CNN_CLEAN_DATASET_TYPE_TEST_NONTYPE}")
print(f"PATH_CNN_CLEAN_OUTPUT                    : {PATH_CNN_CLEAN_OUTPUT}")
print(f"PATH_CNN_CLEAN_OUTPUT_TYPE               : {PATH_CNN_CLEAN_OUTPUT_TYPE}")

### Reset Type

In [ ]:
delete_path(PATH_CNN_CLEAN_DATASET_TYPE)
delete_path(PATH_CNN_CLEAN_OUTPUT_TYPE)

### Checking Folders

In [ ]:
#create_path(PATH_CNN_CLEAN_BASE)
#create_path(PATH_CNN_CLEAN_DATASET)
#create_path(PATH_CNN_CLEAN_DATASET_TYPE)
#create_path(PATH_CNN_CLEAN_DATASET_TYPE_TRAIN)
create_path(PATH_CNN_CLEAN_DATASET_TYPE_TRAIN_TYPE)
create_path(PATH_CNN_CLEAN_DATASET_TYPE_TRAIN_NONTYPE)
#create_path(PATH_CNN_CLEAN_DATASET_TYPE_TEST)
create_path(PATH_CNN_CLEAN_DATASET_TYPE_TEST_TYPE)
create_path(PATH_CNN_CLEAN_DATASET_TYPE_TEST_NONTYPE)
#create_path(PATH_CNN_CLEAN_OUTPUT)
create_path(PATH_CNN_CLEAN_OUTPUT_TYPE)

## Renaming Images

In [ ]:
def rename_images(img_list, file_path, prefix):
    count = 0
    for img in img_list:
        os.rename(os.path.join(file_path, img), os.path.join(file_path, f"{prefix}_{count}.jpg"))
        count += 1
    
train_type_images = next(os.walk(PATH_CNN_CLEAN_DATASET_TYPE_TRAIN_TYPE))[2] #0 - root, 1 - dirs, 2 - files
train_nontype_images = next(os.walk(PATH_CNN_CLEAN_DATASET_TYPE_TRAIN_NONTYPE))[2] #0 - root, 1 - dirs, 2 - files
test_type_images = next(os.walk(PATH_CNN_CLEAN_DATASET_TYPE_TEST_TYPE))[2] #0 - root, 1 - dirs, 2 - files
test_nontype_images = next(os.walk(PATH_CNN_CLEAN_DATASET_TYPE_TEST_NONTYPE))[2] #0 - root, 1 - dirs, 2 - files

rename_images(train_type_images, PATH_CNN_CLEAN_DATASET_TYPE_TRAIN_TYPE, 1)
rename_images(train_nontype_images, PATH_CNN_CLEAN_DATASET_TYPE_TRAIN_NONTYPE, 0)
rename_images(test_type_images, PATH_CNN_CLEAN_DATASET_TYPE_TEST_TYPE, 1)
rename_images(test_nontype_images, PATH_CNN_CLEAN_DATASET_TYPE_TEST_NONTYPE, 0)

log_print("All images has been renamed")

## Extract Features

In [ ]:
le = LabelEncoder()
le.fit([f"non_{TYPE}", TYPE])
# serialize the label encoder to disk
joblib.dump(le, os.path.join(PATH_CNN_CLEAN_OUTPUT_TYPE, "le.pickle"))

# load the VGG16 network and initialize the label encoder
log_print("[INFO] loading network...")
model = VGG16(weights="imagenet", include_top=False)

# loop over the data splits
for split in (["Train", "Test"]):
    # grab all image paths in the current split
    log_print(f"[INFO] processing '{split} split'...")
    p = os.path.join(PATH_CNN_CLEAN_DATASET_TYPE, split)
    imagePaths = list(paths.list_images(p))

    # randomly shuffle the image paths and then extract the class
    # labels from the file paths
    random.shuffle(imagePaths)

    labels = [p.split(os.path.sep)[-2] for p in imagePaths]

    # open the output CSV file for writing
    csv = open(os.path.join(PATH_CNN_CLEAN_OUTPUT_TYPE, split + ".csv"), "w")

    loop_images_extract_features(imagePaths, labels, csv)

    # close the CSV file
    csv.close()
log_print("Finished")

## Train

In [ ]:
# derive the paths to the training and testing CSV files
trainingPath = os.path.join(PATH_CNN_CLEAN_OUTPUT_TYPE, "Train.csv")
testingPath = os.path.join(PATH_CNN_CLEAN_OUTPUT_TYPE, "Test.csv")

# load the data from disk
log_print("[INFO] loading data...")
(trainX, trainY) = load_data_split(trainingPath)
(testX, testY) = load_data_split(testingPath)

# load the label encoder from disk
le = joblib.load(os.path.join(PATH_CNN_CLEAN_OUTPUT_TYPE, "le.pickle"))

# train the model
log_print("[INFO] training model...")
model = LogisticRegression(solver="lbfgs", multi_class="auto")
model.fit(trainX, trainY)

# evaluate the model
log_print("[INFO] evaluating...")
preds = model.predict(testX)
log_print(metrics.classification_report(testY, preds, target_names=le.classes_))
plot_confusion_matrix(testY, preds, np.array(le.classes_), normalize=True)

# serialize the model to disk
log_print("[INFO] saving model...")
plt.savefig(os.path.join(PATH_CNN_CLEAN_OUTPUT_TYPE, "test_normalized.svg"))
joblib.dump(model, os.path.join(PATH_CNN_CLEAN_OUTPUT_TYPE, "model.pickle"))

## Clean

In [ ]:
img = load_img("D:\IC\\tcc_insta\data\\1-Creating_Dataset\Dataset\guastafriends\joao.squinelato\images\\2553959008324384891.jpg", target_size=(224, 224))

In [ ]:
print(imagenet_utils.preprocess_input(np.expand_dims(img_to_array(img), axis=0)))

In [ ]:
BATCH_SIZE = 60
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [ ]:
# load the VGG16 network and initialize the label encoder
log_print("[INFO] loading network...")
vgg16 = VGG16(weights="imagenet", include_top=False)

model = joblib.load(os.path.join(PATH_CNN_CLEAN_OUTPUT_TYPE, "model.pickle"))

list_accounts = get_accounts()

for acc in list_accounts:

    log_print("Processing account: " + acc)

    posts_overview = read_json(os.path.join(PATH_FIRST_DATASET_TYPE, acc, "user_posts_overview.json"))
    df_posts_overview = None
    df_posts_overview = pd.DataFrame(posts_overview["items"])
    df_posts_overview = df_posts_overview.apply(lambda x: [ (x["pk"], image["pk"], os.path.join(PATH_FIRST_DATASET_TYPE, acc, "images", f"{image['pk']}.jpg"), image["url"]) for image in x["images"]], axis=1)
    df_posts_overview = pd.DataFrame(list(itertools.chain.from_iterable(df_posts_overview.to_list())), columns=["pk_item", "pk_image", "path_image", "url"])
    
    print(df_posts_overview)
    tam = len(df_posts_overview.index)
    for offset in range(0, tam, BATCH_SIZE):
        log_print(f"Batch {offset+1}/{round((tam/BATCH_SIZE)+0.5)}")
        log_print("Creating image_values")
        a = df_posts_overview["path_image"].to_list()
        print(imagenet_utils.preprocess_input(np.expand_dims(img_to_array(load_img(a[-1], target_size=(224, 224))), axis=0)))
        df_posts_overview.loc[offset:offset+BATCH_SIZE,'image_values'] = df_batch.apply(lambda row: imagenet_utils.preprocess_input(np.expand_dims(img_to_array(load_img(row["path_image"], target_size=(224, 224))), axis=0)), axis=1)
        
        print(df_posts_overview.loc[offset:offset+BATCH_SIZE,'image_values'])
        
        log_print("Predicting")
        batchImages = np.vstack(df_posts_overview.loc[offset:offset+BATCH_SIZE,'image_values'])
        features = vgg16.predict(batchImages, batch_size=BATCH_SIZE)
        features = features.reshape((features.shape[0], 7 * 7 * 512))
        resul = model.predict(features)
        df_posts_overview.loc[offset:offset+BATCH_SIZE,'predicted_label'] = resul

    

In [ ]:
log_print("Images:")
    
    
    
    

                imagePath = all_image_paths[j]

                image_model = 
                image_model = 

                # preprocess the image by (1) expanding the dimensions and
                # (2) subtracting the mean RGB pixel intensity from the
                # ImageNet dataset
                image_model = 
                image_model = imagenet_utils.preprocess_input(np.expand_dims(img_to_array(load_img(imagePath, target_size=(224, 224))), axis=0))

                batchImages.append(image_model)
        
        batchImages = np.vstack(batchImages)
        features = vgg16.predict(batchImages, batch_size=BATCH_SIZE)
        features = features.reshape((features.shape[0], 7 * 7 * 512))

        resul = model.predict(features)
        
        
        
        
        
        
        
        
        
        
        posts_overview_clean = {}
        posts_overview_clean["collected_at"] = posts_overview["collected_at"]
        posts_overview_clean["num_results"] = 0
        posts_overview_clean["items"] = []
        posts_overview_clean["media_count_total"] = 0

        
        
        for j in range(0, BATCH_SIZE):
            soma = j+i
            if soma <= tam:
                if resul[j] == '0':
            
            
            
            
            
            
            
            
            
            item_copy = item.copy()
            item_copy["media_count"] = 0
            item_copy["images"] = []

                imagePath = os.path.join(PATH_FIRST_DATASET_TYPE, acc, "images", str(all_images[j]['pk']) + ".jpg")
                log_print(str(count_image) + "/" + str(posts_overview["media_count_total"]) + " - " + imagePath)
                count_image += 1

                try_count = 0
                try_again = True
                while try_again:
                    try:
                        
                        

                        

                        if resul[0] == '0':
                            log_print(TYPE + "\n")
                            item_copy["images"].append(image)
                            item_copy["media_count"] += 1
                            posts_overview_clean["media_count_total"] += 1
                        else:
                            log_print("non_" + TYPE + "\n")
                        try_again = False
                    except Exception as err:
                        log_print("Error: " + str(err))
                        log_print("Downloading image again: " + image['url'])                  
                        download_image(image['url'], imagePath)
                        
                        try_count += 1
                        if try_count > 3:
                            os.remove(imagePath)
                            write_file(os.path.join(folder_path, "deleted_images_path.txt"), [imagePath, "\n"])
                            try_again = False

            if item_copy["media_count"] > 0:
                posts_overview_clean["items"].append(item_copy)
                posts_overview_clean["num_results"] += 1

    if posts_overview_clean["num_results"] > 0:
        create_path(os.path.join(PATH_CLEAN_DATASET_TYPE, acc))
        write_json(os.path.join(PATH_CLEAN_DATASET_TYPE, acc, "user_posts_overview_clean.json"), posts_overview_clean)

    print("**********************************************\n")
            
log_print("Finished")

## Moving Images

In [ ]:
list_accounts = get_accounts()

for acc in list_accounts:
    
    log_print(f"Processing account {acc}")
    
    shutil.copyfile(os.path.join(PATH_FIRST_DATASET_TYPE, acc, "user_info.json"), os.path.join(PATH_CLEAN_DATASET_TYPE, acc, "user_info.json"))

    create_path(os.path.join(PATH_CLEAN_DATASET_TYPE, acc, "images"))

    overview_clean = read_json(os.path.join(PATH_CLEAN_DATASET_TYPE, acc, "user_posts_overview_clean.json"))

    images_moved = len(list(next(os.walk(os.path.join(PATH_CLEAN_DATASET_TYPE, acc, "images")))[2])) - 1 #0 - root, 1 - dirs, 2 - files
    if images_moved <= 0:
        images_moved = 0
    for item in overview_clean["items"]:
        for image in item["images"]:
            if not os.path.exists(os.path.join(PATH_CLEAN_DATASET_TYPE, acc, "images", str(image["pk"]) + ".jpg")):
                shutil.move(os.path.join(PATH_FIRST_DATASET_TYPE, acc, "images", str(image["pk"]) + ".jpg"), os.path.join(PATH_CLEAN_DATASET_TYPE, acc, "images", str(image["pk"]) + ".jpg"))
                images_moved += 1
                log_print(f"{images_moved}/{overview_clean['media_count_total']}")
    print("**********************************************\n")
            
log_print("Finished")

# Processing Dataset

In [ ]:
df2

## Similary Accounts

In [ ]:
import itertools

In [ ]:
def get_users_comparison(i_followers, j_followers):
    if i_followers >= j_followers:
        diff_radius = i_followers * FOLLOWERS_RADIUS // 100
    else:
        diff_radius = j_followers * FOLLOWERS_RADIUS // 100

    diff_abs = abs(i_followers - j_followers)
    
    return { "comparison": diff_abs <= diff_radius,
           "diff_abs": diff_abs,
           "diff_radius": diff_radius }
    
def get_items_comparison(i_likes, j_likes):
    if i_likes >= j_likes:
        diff_radius = i_likes * LIKES_RADIUS // 100
    else:
        diff_radius = j_likes * LIKES_RADIUS // 100        
        
    diff_abs = abs(i_likes - j_likes)
    
    if diff_abs >= diff_radius:
        i_label = "igual"
        j_label = "igual"

        if i_likes > j_likes:
            i_label = "inte"
            j_label = "desinte"
        elif i_likes < j_likes:
            i_label = "desinte"
            j_label = "inte"
    else:
        i_label = None
        j_label = None
    
    return { "comparison": diff_abs >= diff_radius,
           "diff_abs": diff_abs,
           "diff_radius": diff_radius,
           "i_label": i_label,
           "j_label": j_label}
            

In [ ]:
list_accounts = get_accounts()
random.shuffle(list_accounts)
tam = len(list_accounts)

for i in range(tam-1):
    
    user_info_I = read_json(os.path.join(PATH_CLEAN_DATASET_TYPE, list_accounts[i], "user_info.json"))
        
    for j in range(i+1, tam):
        comparisons_list = []

        user_info_J = read_json(os.path.join(PATH_CLEAN_DATASET_TYPE, list_accounts[j], "user_info.json"))

        log_print(f"Comparando conta {i+1}-[{list_accounts[i]}] com conta {j+1}-[{list_accounts[j]}] / Total: {tam} contas.")

        users_comparison = get_users_comparison(user_info_I["follower_count"], user_info_I["follower_count"])
        if users_comparison["comparison"]:

            user_posts_overview_clean_I = read_json(os.path.join(PATH_CLEAN_DATASET_TYPE, list_accounts[i], "user_posts_overview_clean.json"))
            user_posts_overview_clean_J = read_json(os.path.join(PATH_CLEAN_DATASET_TYPE, list_accounts[j], "user_posts_overview_clean.json"))

            random.shuffle(user_posts_overview_clean_I["items"])
            random.shuffle(user_posts_overview_clean_J["items"])

            for item_I in user_posts_overview_clean_I["items"]:
                for item_J in user_posts_overview_clean_J["items"]:

                    items_comparison = get_items_comparison(item_I["like_count"], item_J["like_count"])
                    if (items_comparison["comparison"]):

                        row = {
                             'type': TYPE,
                             'i_username': list_accounts[i],
                             'i_follower_count': user_info_I["follower_count"],
                             'j_username': list_accounts[j],
                             'j_follower_count': user_info_J["follower_count"],
                             'diff_followers_abs': users_comparison["diff_abs"],
                             'diff_followers_radius': users_comparison["diff_radius"],
                             'i_pk': item_I["images"][0]["pk"],
                             'i_like_count': item_I["like_count"],
                             'j_pk': item_J["images"][0]["pk"],
                             'j_like_count': item_J["like_count"],
                             'diff_likes_abs': items_comparison["diff_abs"],
                             'diff_likes_radius': items_comparison["diff_radius"],
                             'i_label': items_comparison["i_label"],
                             'j_label': items_comparison["j_label"]
                        }

                        comparisons_list.append(row)

            if len(comparisons_list) > 0:
                log_print(f"Saving {len(comparisons_list)} new rows")
                df = pd.DataFrame(comparisons_list)
                df.to_parquet(path=PATH_SIMILARY_ACCOUNTS_TYPE, partition_cols="i_username")

## Label Counter

In [ ]:
df_similary_accounts = pd.read_parquet(path=PATH_SIMILARY_ACCOUNTS_TYPE)

In [ ]:
df_i = df_similary_accounts.loc[:, ["type", "i_username", "i_follower_count", "i_pk", "i_like_count", "i_label"]].copy()
df_j = df_similary_accounts.loc[:, ["type", "j_username", "j_follower_count", "j_pk", "j_like_count", "j_label"]].copy()

columns = ["type", "username", "follower_count", "pk", "like_count", "label"]
df_i.columns = columns
df_j.columns = columns

df_united = pd.concat([df_i, df_j], axis="index")

df_label_counter_aux = df_united.groupby(columns).size().reset_index(name="count").sort_values(["pk","count"]).drop_duplicates(subset=['pk'], keep='first').copy()
df_label_counter_aux

In [ ]:
if RANDOM_IGUAL:
    df_label_counter = df_label_counter_aux.copy()
    df_label_counter.loc[df_label_counter['label'] == 'igual', 'label'] = [ random.choice(["inte", "desinte"]) for k in df_label_counter.loc[df_label_counter['label'] == 'igual'].index ]
else:
    df_label_counter = df_label_counter.loc[df_label_counter['label'] != 'igual'].copy()
df_label_counter

In [ ]:
df_label_counter.to_parquet(path=PATH_LABEL_COUNTER_TYPE, partition_cols="label")

# Classifying Dataset

## CNN Classify Configs

In [ ]:
PATH_CNN_CLASSIFY_BASE                       = os.path.join(PATH_4_CLASSIFYING_DATASET, "CNN_Classify")
PATH_CNN_CLASSIFY_OUTPUT                     = os.path.join(PATH_CNN_CLASSIFY_BASE, "CNN_Output")
PATH_CNN_CLASSIFY_OUTPUT_TYPE                = os.path.join(PATH_CNN_CLASSIFY_OUTPUT, TYPE)

print(f"PATH_CNN_CLASSIFY_BASE                      : {PATH_CNN_CLASSIFY_BASE}")
print(f"PATH_CNN_CLASSIFY_OUTPUT                    : {PATH_CNN_CLASSIFY_OUTPUT}")
print(f"PATH_CNN_CLASSIFY_OUTPUT_TYPE               : {PATH_CNN_CLASSIFY_OUTPUT_TYPE}")

### Reset Type

In [ ]:
delete_path(PATH_CNN_CLASSIFY_OUTPUT_TYPE)

### Checking Folders

In [ ]:
#create_path(PATH_CNN_CLASSIFY_BASE)
#create_path(PATH_CNN_CLASSIFY_OUTPUT)
create_path(PATH_CNN_CLASSIFY_OUTPUT_TYPE)

## Extract Features

In [ ]:
df_label_counter = pd.read_parquet(path=PATH_LABEL_COUNTER_TYPE)

In [ ]:
count_interessantes = len(df_label_counter.loc[df_label_counter['label'] == 'inte'].index)
count_desinteressantes = len(df_label_counter.loc[df_label_counter['label'] == 'desinte'].index)

count_part_interessantes = count_interessantes // CV
count_part_desinteressantes = count_desinteressantes // CV
count_total = (count_part_interessantes + count_part_desinteressantes) * CV

print(f"Type: {TYPE}")
print(f"Interessantes: {count_part_interessantes * CV}")
print(f"Desinteressantes: {count_part_desinteressantes * CV}")
print(f"Total: {count_total}")

In [ ]:
df_interessantes = df_label_counter.loc[df_label_counter['label'] == 'inte'].sample(n = count_interessantes).copy()
df_desinteressantes = df_label_counter.loc[df_label_counter['label'] == 'desinte'].sample(n = count_desinteressantes).copy()

In [ ]:
df_data = pd.concat([df_interessantes, df_desinteressantes], axis="index")
df_data = df_data.sample(frac=1).reset_index(drop=True)
df_data['imagePath'] = df_data.apply(lambda row: os.path.join(PATH_CLEAN_DATASET_TYPE, row.username, 'images', str(row.pk) + '.jpg'), axis=1)

imagePaths = df_data.loc[:, 'imagePath'].copy().values.tolist()
labels = df_data.loc[:, 'label'].copy().values.tolist()

In [ ]:
le = LabelEncoder()
le.fit(["desinte", "inte"])
# serialize the label encoder to disk

joblib.dump(le, os.path.join(PATH_CNN_CLASSIFY_OUTPUT_TYPE, "le.pickle"))

print("[INFO] loading network...")
model = VGG16(weights="imagenet", include_top=False)

print("[INFO] processing...")
# open the output CSV file for writing
csv = open(os.path.join(PATH_CNN_CLASSIFY_OUTPUT_TYPE, "data.csv"), "w")

loop_images_extract_features(imagePaths, labels, csv)

# close the CSV file
csv.close()

## Train With Cross-Validation

In [ ]:
# derive the paths to the training and testing CSV files
dataPath = os.path.join(PATH_CNN_CLASSIFY_OUTPUT_TYPE, "data.csv")

# load the data from disk
print("[INFO] loading data...")
(X, y) = load_data_split(dataPath)
# load the label encoder from disk
le = joblib.load(os.path.join(PATH_CNN_CLASSIFY_OUTPUT_TYPE, "le.pickle"))

# train the model
print("[INFO] training model...")
predicted = model_selection.cross_val_predict(LogisticRegression(solver="lbfgs", multi_class="auto"), X, y, cv=CV)

# evaluate the model
print("[INFO] evaluating...")
plot_confusion_matrix(y, predicted, np.array(le.classes_), normalize=True)
#plt.show()
result = metrics.classification_report(y, predicted, target_names=le.classes_)
print(result)

# save the result to disk
print("[INFO] saving result...")
plt.savefig(os.path.join(PATH_CNN_CLASSIFY_OUTPUT_TYPE, "test_normalized.svg"))
write_file(os.path.join(PATH_CNN_CLASSIFY_OUTPUT_TYPE, "result.txt"), [result, "\n"])